# Simple Classifier
In der vorherigen Teilaufgabe haben Sie einiges über die Struktur der Daten gelernt. Nun sollen Sie dieses Wissen einsetzen um eine erste Vorhersage auf einem Testdatensatz zu treffen. Im folgenden werden die notwendigen Imports getätigt und das Dataset eingelesen.

### Imports

In [38]:
import numpy as np
import csv as csv
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import random
%matplotlib inline

### Daten einlesen

In [39]:
DATA_FILE = './Data/original_titanic.csv'
df = pd.read_csv(DATA_FILE)

### Datenlücken interpolieren

Gerne können Sie zum Füllen der Datenlücken auch Ihre Implementierung aus der vorherigen Teilaufgabe einsetzen.

In [40]:
def prepareData(df):
    #to set a selection of a data frame please dont use the chained [] operator, but the .loc operator
    #(see http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy for details)
    
    #minimal working example to set all null age values to 42 - probably you can do better :-)
#     df.loc[df.Age.isnull(),'Age'] = 42
#     return df

    df.loc[:,'Age'].interpolate(method='pad', inplace= True) #using existing values to fill NaN values
    
prepareData(df)

### Datensatz stochastisch verändern

Es wird der komplette Datensatz durchmischt. Finden Sie hierfür geeignete Methoden (Python, Numpy, Pandas). Geben Sie die ersten fünf Zeilen des durchmischten Datensatzes aus. Weisen Sie nach, dass lediglich die Reihenfolge der Daten geändert wurde und <b>nicht(!)</b> die Merkmals-Spalten.

In [41]:
print("Original: {}:\n".format(df[df['PassengerId']==5]))
df_shuffled = df.sample(frac = 1).reset_index(drop=True) 
print("Suffled: {}:\n".format(df_shuffled[df_shuffled['PassengerId']==5]))

print(df_shuffled[:5])

Original:    PassengerId  Survived  Pclass  \
4            5         0       1   

                                              Name     Sex   Age  SibSp  \
4  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female  25.0      1   

   Parch  Ticket    Fare    Cabin Embarked                        Home-Dest  
4      2  113781  151.55  C22 C26        S  Montreal, PQ / Chesterville, ON  :

Suffled:      PassengerId  Survived  Pclass  \
461            5         0       1   

                                                Name     Sex   Age  SibSp  \
461  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female  25.0      1   

     Parch  Ticket    Fare    Cabin Embarked                        Home-Dest  
461      2  113781  151.55  C22 C26        S  Montreal, PQ / Chesterville, ON  :

   PassengerId  Survived  Pclass                          Name   Sex   Age  \
0         1006         0       3           McEvoy, Mr. Michael  male  33.0   
1          632         0       3   Andersson, Mr.

### Aufteilung in Trainings- und Testdatensatz

Teilen Sie den stochastisch veränderten Datensatz in Trainings- und Testdatensatz mit der Verteilung von 80% (Train) zu 20% (Test) auf. Geben Sie die Länge der Datensätze aus.

In [42]:
len_all = len(df_shuffled)
len_train = round(len_all * 0.8, 0)
len_test = round(len_all * 0.2, 0)

df_train = df_shuffled[: int(len_train)]
df_test  = df_shuffled[: int(len_test)]

### Performance von Machine Learing Algorithmen

Die Performance von Machine Learing Algorithmen kann unter anderem anhand der Korreklassifizierungsrate (engl. Accuracy) ermittelt werden. Diese wird über die Verwechslungsmatrix (engl. Confusion matrix) (siehe Abbildung) wie folgt definiert: <br>

TP = True Positive <br>
TN = True Negative <br>
FP = False Positive <br>
FN = False Negative <br>
ACC = Accuracy <br>

$ACC = \frac{(TP + TN)} {(TP+TN+FP+FN)}$

<img src="./Figures/Confusion-Matrix.png" alt="drawing" style="width:300px;"/>



### Implementierung des Algorithmus

Schreiben Sie eine Funktion um die Klasse *Survived*, mit ihren beiden Ausprägungen 0 und 1, anhand von einem oder mehreren Merkmalen vorherzusagen. Verwenden Sie dazu ein einfaches Modell in welchem Sie ihr bisheriges Wissen über den Datensatz einsetzen.

In [62]:
def predict(passenger):
    # anhand der in a erhaltenen Erkenntnisse, über Trennungswirksame merkmale
    #wahrscheinlicher -> überlebt wenn in klasse 1, unwahrscheinlicher wenn in klasse 3
    # unwahrscheinlich, wenn älter als 65 dass tot
    #wenn frau wahrschienlicher
    # TODO: implement
    if passenger.Age > 65:
        return 0
        
    if passenger.Sex == "female":
        if passenger.Pclass != 3:
            return 1
       
        r = random.randint(0,100)
        if r <50: 
            return 0
        if r >= 50:
            return 1
            
    else:
        if passenger.Pclass != 3:
            return 1
       
        return 0

### Testen des Algorithmus

Testen Sie die predict() Methode mit den Datenobjekten aus dem Testdatensatz. Ermitteln sie hierzu die Korreklassifizierungsrate.

In [50]:
# TODO: implement
# for one data set-> Testing predict()

rand = random.randint(0, len(df_test))
print("Sample to predict:\n{}".format(df_test.iloc[rand]))
prediction = predict(df_test.iloc[rand])
print("Prediction: {}".format("Survivor" if prediction== 1 else "Dead"))

Sample to predict:
PassengerId                                     844
Survived                                          0
Pclass                                            3
Name           Hagland, Mr. Konrad Mathias Reiersen
Sex                                            male
Age                                              17
SibSp                                             1
Parch                                             0
Ticket                                        65304
Fare                                        19.9667
Cabin                                           NaN
Embarked                                          S
Home-Dest                                       NaN
Name: 144, dtype: object
Prediction: Dead


In [67]:
predictions = []
labels = []
for idx,row in df_test.iterrows():
    predictions.append(predict(row))
    labels.append(row.loc['Survived'])
    
tp = 0
fp = 0
fn = 0
tn = 0
for idx, val in enumerate(labels):
    pred = predictions[idx]
    if val == predictions[idx]:
        if val == 1:
            tp +=1
        else:
            tn += 1
            
    else: 
        if val == 1:
            fn +=1
        else:
            fp += 1
        
                
print("True Positives: %d" %tp)   
print("True Negatives: %d" %tn)

print("False Positives: %d" %fp)
print("False Negatives: %d" %fn)

acc = (tp + tn) / (tp + tn + fp + fn)
print("Accurency: %f" %acc)

True Positives: 85
True Negatives: 91
False Positives: 68
False Negatives: 18
Accurency: 0.671756
